# directNeo4jImporter

## *! not working yet ! *

This file is used to directly push blocks and transactions (incl. addresses to neo4j)

In [ ]:
# parameter handed over
kafka_topics =   {
    'transaction': 'transactions', 
    'block': 'blocks'
}
node_labels = {
    'original': {
        'block'       : 'Block',
        'transaction' : 'Transaction',
        'address'     : 'Address'
    },
    'test': {
        'block'       : 'Block_test',
        'transaction' : 'Transaction_test',
        'address'     : 'Address_test'
    }
}
evaluate_original = False

In [ ]:
# imports
import sys
from cypher_queries import *
sys.path.append("../")
from neo4jConnector import startNeo4jSession

In [ ]:
driver, session = startNeo4jSession() 


In [ ]:
queryBlocks = getCypherQueries(topic = kafka_topics['block'], 
                               matchOnAddress = False, 
                               getTemplate = False, 
                               node_labels = node_labels,
                               evaluate_original = False)

queryTransactions = getCypherQueries(topic = kafka_topics['transaction'], 
                               matchOnAddress = False, 
                               getTemplate = False, 
                               node_labels = node_labels,
                               evaluate_original = False)
print(queryTransactions)

In [ ]:
def directNeoImport(startBlock, endBlock):

    driver, session = startNeo4jSession() 
    
    for blockheight in range(startBlock,endBlock,getInput):
        try:
            # retrival of block + transfer to the respective topic
            blockhash = getBlockHash(blockheight)
            data, block = getblock(blockhash)
            
            # push to neo4j
            
            
        except:
            print('Error handling block at height' + str(blockheight) + 'with blockhash' + str(blockhash))
    

        # retrival of transactions + transfer to the respective topic
        for txid in block['tx'][:]:
            try:
                tx = gettx(txid,block,getInput)
                
                # push to neo4j
                
                
            except:
                print('Error handling transactions at block' + str(blockheight) + ' with transaction id' + str(txid['txid']))

    return 

In [ ]:
test = {'txid': '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098', 'block_hash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 'block_date': '2009-01-09', 'outDegree': 1, 'inDegree': 1, 'outSum': 5000000000, 'inSum': 5000000000, 'input_list': [{'addr': 'coinbase', 'val': 5000000000}], 'output_list': [{'addr': '12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX', 'val': 5000000000, 'outNr': 0}]}

#run_tx()

In [ ]:
def run_tx(tx, event):
    tx.run(queryTransactions, 
           txid         = event["txid"], 
           output_list  = event["output_list"], 
           block_hash   = event["block_hash"], 
           inDegree     = event["inDegree"], 
           outDegree    = event["outDegree"], 
           outSum       = event["outSum"],
           inSum        = event["inSum"],
           input_list   = event["input_list"], 
           block_date   = event["block_date"])


with driver.session() as session:
    session.write_transaction(run_tx, test)

In [ ]:
session.run('Match (n:Block_test) return n LIMIT 5').values()

In [ ]:
#queryTransactions = getCypherQueries(topic = kafka_topics['transaction'], 
                               evaluation = False, 
                               matchOnAddress = False, 
                               getTemplate = False, 
                               node_labels = node_labels,
                               evaluate_original = True)
print(queryTransactions)

In [ ]:
queryTransactions